# Writing a new connection plugin

Connection plugins are installed as separate packages. Those package must register them self as an entry point for the discovery. ``InitNornir()`` discovers and register all plugins under the entry point group ``nornir.plugins.connections`` automatically.

## Setuptools

If you use the setuptools you can use the ``entry_points`` argument in ``setup()`` to register the plugin for discovery. Add the following in your ``setup.py``.

```python
setup(
    ...
    entry_points={'nornir.plugins.connections': 'nornir_demo.demo = nornir_demo.connections:Demo'},
    ...
)
```

## Poetry

If you use poetry like the nornir project you have to add the following in the ``pyproject.toml`` file to register the plugin.

```python
[tool.poetry.plugins."nornir.plugins.connections"]
"nornir_demo.demo" = "nornir_demo.connections:Demo"
```

## Poetry demo setup

In [1]:
!poetry new nornir_demo

Created package nornir_demo in nornir_demo


In [2]:
!tree -a -n nornir_demo

nornir_demo
├── README.rst
├── nornir_demo
│   └── __init__.py
├── pyproject.toml
└── tests
    ├── __init__.py
    └── test_nornir_demo.py

2 directories, 5 files


In [3]:
!cd nornir_demo && mkdir nornir_demo/connections

In [4]:
!cd nornir_demo && touch nornir_demo/connections/__init__.py

In [5]:
!tree -a -n nornir_demo

nornir_demo
├── README.rst
├── nornir_demo
│   ├── __init__.py
│   └── connections
│       └── __init__.py
├── pyproject.toml
└── tests
    ├── __init__.py
    └── test_nornir_demo.py

3 directories, 6 files


## Connection plugin implementation

The connection class must inherit the abstract ``ConnectionPlugin`` class from ``nornir.core.connections`` and need to have a implementation of the methods ``open`` and ``close``. Assign the underlaying connetion to ``self.connection``.

In [6]:
%cat writing_a_new_connection_plugin/__init__.py

from typing import Any, Dict, Optional

from nornir.core.configuration import Config
from nornir.core.connections import ConnectionPlugin


CONNECTION_NAME = "nornir_demo.demo"


class Demo(ConnectionPlugin):
    """
    This plugin is a demo plugin to show how to set up a new project
    """

    def open(
        self,
        hostname: Optional[str],
        username: Optional[str],
        password: Optional[str],
        port: Optional[int],
        platform: Optional[str],
        extras: Optional[Dict[str, Any]] = None,
        configuration: Optional[Config] = None,
    ) -> None:
        """
        Connect to the device and populate the attribute :attr:`connection` with
        the underlying connection
        """
        pass


    def close(self) -> None:
        """Close the connection with the device"""
        pass


Don't forget to register the plugin in the ``pyproject.toml``

In [7]:
%cat writing_a_new_connection_plugin/pyproject.toml

[tool.poetry]
name = "nornir_demo"
version = "0.1.0"
description = "Nornir demo connection plugin"
authors = ["Jane Doe <jane@doe.loc>"]
license = "Apache-2.0"

[tool.poetry.plugins."nornir.plugins.connections"]
"nornir_demo.demo" = "nornir_demo.connections:Demo"

[tool.poetry.dependencies]
python = "^3.6"

[tool.poetry.dev-dependencies]
pytest = "^5.2"

[build-system]
requires = ["poetry>=0.12"]
build-backend = "poetry.masonry.api"


Nornir should be added as a dependency

In [8]:
!cd nornir_demo && poetry add nornir^3.0.0 -q

## Test entry point registration

To test we need to install the new package. (If you use the setuptool you also need to install it with ``setup.py install`` or ``setup.py develop``)

In [9]:
!cd nornir_demo && poetry install -q

In [10]:
!cd nornir_demo && poetry shell -q

In [11]:
import pkg_resources

entry_point = pkg_resources.iter_entry_points('nornir.plugins.connections')

for ep in entry_point:
    if 'nornir_demo.demo' in str(ep):
        print('Install successful')

Install successful


## Documentation

Nornir plugins should be well documented. After the plugin is released and published add it to the list of available plugins ([nornir.tech](https://nornir.tech/nornir/plugins/)). The list is populated from the GitHub repo [
nornir.tech.src](https://github.com/nornir-automation/nornir.tech.src/blob/master/data/nornir/plugins.yaml). Please create a pull request.

```yaml
---
connections:
    - url: https://github.com/nornir-automation/nornir_demo
      name: demo
      maintainer: "[Jane Doe](https://github.com/janedoe)"
      description: |
          Project to demonstrate how to create a connection plugin
```